In [138]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [139]:
import json 
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col, avg
from pyspark.sql import SparkSession 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [140]:
rdd_data = sc.textFile('../RecSysMovies/ml-100k/train.csv')
rdd_data = rdd_data.map(lambda x: x.split('\t')).map(lambda x: (int(x[0]),int(x[1]),int(x[2])))
rdd_data.cache()

PythonRDD[1410] at RDD at PythonRDD.scala:48

In [141]:
df_train = spark.read.csv('/labs/lab09data/train.csv',header=True,inferSchema=True)
df_test = spark.read.csv('/labs/lab09data/test.csv',header=True,inferSchema=True)

In [142]:
df_train.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [143]:
df_test.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [144]:
avg = df_train.select(F.avg("rating")) #df_train.agg(F.mean('rating').alias("Mr")).rdd.flatMap(lambda x: x).collect()
avg = avg.toPandas().iloc(0)[0][0]
avg

3.5218638466233507

In [173]:
als = ALS(maxIter=10, regParam=0.1, seed=5L, rank=8, userCol="userId", itemCol="movieId", ratingCol="rating")

In [174]:
model = als.fit(df_train)

In [175]:
pred = model.transform(df_test)

In [176]:
pred = pred.fillna(avg)

In [177]:
pred.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|  3855|    148|   0.0|  2.461945|
| 94231|    148|   0.0|  3.334913|
|178586|    148|   0.0|  2.597012|
|155572|    148|   0.0| 2.9842262|
|145363|    463|   0.0| 3.6137238|
+------+-------+------+----------+
only showing top 5 rows



In [178]:
pred = pred.selectExpr('userId', 'movieId', 'prediction as rating')

In [179]:
pred = pred.sort(pred.userId,pred.movieId)

In [183]:
pred.coalesce(1).write.mode("overwrite").option("header", "true").csv("lab09s.csv")
!hadoop fs -getmerge lab09s.csv ~/lab09s.csv